# Exercice 1. CNN for object classification

Le jeu de données Toy Dataset - v7 2023-04-19 1:21am est fourni par Roboflow (https://universe.roboflow.com/tips-wis6y/toy-dataset-bfq0d). Vous en trouverez une copie sur la page Moodle.

Les données brutes se trouvent dans le sous-dossier « images ». Les données d'étiquetage se trouvent dans le sous-dossier « labels ».

Le format des données d'étiquetage est le suivant :

    categ, cx, cy, w, h

où categ représente le numéro de catégorie, cx et cy les coordonnées du centre de l'objet (sur une échelle de 0 à 1), et w et h ses valeurs horizontale et verticale (exprimées en pourcentage de la largeur et de la hauteur). (cx, cy, w, h) constitue le cadre englobant de l'objet. Cette représentation est conforme au format YOLO8.

Afin d'exploiter tous les objets présents dans les images, lors du chargement du jeu de données, nous pouvons extraire les images des objets à partir des images originales en les recadrant à l'intérieur de leur cadre englobant.

Deux configurations sont présentées ci-dessous :

    all objects versus all objects (load_objects)
    one object versus all (load_objects_as_binary_problem)

**Q1** Vous pouvez utiliser le code tel quel, mais vous pouvez également le modifier afin d'obtenir un équilibre entre les classes, notamment pour la classification binaire. Pourquoi pensez-vous que cela peut être intéressant d'équilibrer entre les classes ?

__Réponse__:

In [ ]:
import os
import csv
import cv2
import numpy as np


train_path="train/"
test_path="test/"

def load_objects(imgs_path,max_samples=np.iinfo(np.int32)):
    y_obj=[]
    x_obj=[]
    imgs_files = os.listdir( imgs_path+"images/" )
    for i,img_file in enumerate(imgs_files):
        if (len(x_obj)==max_samples):
            break
        label_file=img_file[:-4]+".txt"
        img_init=cv2.imread(imgs_path+"images/"+img_file)
        labels=csv.reader(open(imgs_path+"labels/"+label_file,"r"),delimiter=' ')
        rows = list(labels)
        if len(rows)>0:
            for j,row in enumerate(rows):
                if (len(x_obj)==max_samples):
                    break
                y_obj.append(row[0])
                bbox=np.array(row[1:],dtype=np.float32) if row[0]==1 else np.array(row[1:],dtype=np.float32)
                w=int(bbox[2]*img_init.shape[0]*2)
                h=int(bbox[3]*img_init.shape[1]*2)
                x0=max(0,int(math.trunc(bbox[0]*img_init.shape[0]-w/2)))
                y0=max(0,int(math.trunc(bbox[1]*img_init.shape[1]-h/2)))
                x1=min(img_init.shape[0],int(math.trunc(x0+w)))
                y1=min(img_init.shape[1],int(math.trunc(y0+h)))
                x_obj.append(np.copy(img_init[y0:y1,x0:x1]))
    return x_obj,y_obj
                
def load_objects_as_binary_problem(imgs_path,max_samples=np.iinfo(np.int32)):
    y_obj=[]
    x_obj=[]
    imgs_files = os.listdir( imgs_path+"images/" )
    for i,img_file in enumerate(imgs_files):
        if (len(x_obj)==max_samples):
            break
        label_file=img_file[:-4]+".txt"
        #print(i,img_file,label_file)
        img_init=cv2.imread(imgs_path+"images/"+img_file)
        labels=csv.reader(open(imgs_path+"labels/"+label_file,"r"),delimiter=' ')
        rows = list(labels)
        if len(rows)>0:
            for j,row in enumerate(rows):
                if (len(x_obj)==max_samples):
                    break
                if (row[0]=='4'):
                    y_obj.append(int(row[0]))
                else:
                    y_obj.append(0)
                bbox=np.array(row[1:],dtype=np.float32) if row[0]==1 else np.array(row[1:],dtype=np.float32)
                w=int(bbox[2]*img_init.shape[0])
                h=int(bbox[3]*img_init.shape[1])
                x0=max(0,int(math.trunc(bbox[0]*img_init.shape[0]-w/2)))
                y0=max(0,int(math.trunc(bbox[1]*img_init.shape[1]-h/2)))
                x1=min(img_init.shape[0],int(math.trunc(x0+w)))
                y1=min(img_init.shape[1],int(math.trunc(y0+h)))
                x_obj.append(img_init[y0:y1,x0:x1].copy())
    return x_obj,y_obj

def load_objects_as_binary_problem_balanced(imgs_path,max_positivesamples=np.iinfo(np.int32)):
    return [],[]

## Architectures légères

Essayez des architectures simples combinant une ou deux couches de convolution et de pooling, suivies d'une ou deux couches denses.

Pour la classification binaire, la dernière couche ne comporte qu'un seul neurone.

Pour la classification multicatégorielle, la dernière couche comporte autant de neurones que de catégories. Le neurone gagnant est celui qui a la valeur de sortie la plus élevée. Un encodage des étiquettes catégorielles est nécessaire pour préparer les données d'entraînement (voir `tf.keras.utils.to_categorical`).

Vous pouvez choisir le framework (PyTorch ou TensorFlow) avec lequel vous êtes à l'aise.

Vous pouvez travailler en local ou sur Google Colab.

Vous trouverez ci-dessous un exemple de code TensorFlow incomplet pour résoudre ce problème, mais vous pouvez le remplacer complétement par une solution de votre choix.

In [3]:
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import sklearn.cluster as skc
import sklearn.svm as svm

#... 
#x_train,y_train = load_objects_XXX(train_path,1000)
#plt.imshow(layout_images(x_train[:4]))

#preparing the network input tensor
#inputs = tf.keras.Input(shape=x_train.shape[1:])
#preparing the model

# Convolutional Layer #1
#conv1 = tf.keras.layers.Conv2D(
#      filters=64,
#      kernel_size=[5, 5],
#      padding="valid",
#      activation=tf.nn.relu)(inputs)
# Pooling Layer #1
#pool1 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(conv1)
#...
#flat = tf.keras.layers.Flatten()(...)
#mlp = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)(flat)
#output_classif = ...(mlp)

#model = tf.keras.Model(inputs=inputs, outputs=output_classif)

#catloss=tf.keras.losses.CategoricalCrossentropy()
#binloss=tf.keras.losses.CategoricalCrossentropy()

#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#              loss=...,metrics=['accuracy'])

#model.summary()


In [4]:
#model.fit(x=x_train, y=y_train, batch_size=32, epochs=2)

**Q2** Évaluez le modèle sur les ensembles d'entraînement et de test pour les deux modalités de classification (binaire et multiclasse).
Précisez les performances en train et test pour les différentes configurations essayées. 

__Réponse__:


**Q3** Que se passe-t-il si vous changez de modalité de classification ? Faut-il réentraîner entièrement le modèle ? Quelles parties peuvent être conservées ?

__Réponse__ :

**Q4** Avez-vous des remarques concernant la classification d'objets par BoW (TP2) par rapport à la classification d'objets par CNN ?

__Réponse__ :

## Transfer learning (optionnel et pas nécessaire pour la suite)

Dans cette partie, nous allons refaire le même exercice en réutilisant un modèle existant, tel que MobileNet, qui a déjà été entraîné.

Une vaste collection de modèles pré-entraînés est disponible dans des frameworks comme TensorFlow.

Le code ci-dessous utilise MobileNet, mais vous pouvez facilement changer de modèle en instanciant celui de votre choix.

Pour cette partie, il est recommandé d'utiliser Google Colab ou Grid5K.
Dans ce cas, vous devrez également envisager de télécharger les données sur votre Drive ou votre répertoire personnel Grid5K.

In [5]:
#load and prepare the data
#the x_train should be ideally reshaped to (_,224,224,3)

#from tensorflow.keras import layers, models
#from tensorflow.keras.applications.mobilenet import MobileNet


#base_model=MobileNet(weights="imagenet", include_top=False, input_shape=x_train[0].shape)
#include_top is False in order to drop the default Dense Layers and to be able to add your own
#base_model.trainable = False ## Not trainable weights

#flat=layers.Flatten()(base_model)
#mlp=tf.keras.layers.Dense(units=256, activation=tf.nn.relu)(flatten_layer)
#...
#output_classif = ...(mlp)

#model = tf.keras.Model(inputs=inputs, outputs=output_classif)

#catloss=tf.keras.losses.CategoricalCrossentropy()
#binloss=tf.keras.losses.BinaryCrossentropy()

#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#              loss=...,metrics=['accuracy'])

#model.summary()

**Q5** Évaluez le modèle sur les ensembles d'entraînement et de test pour les deux modalités de classification (binaire et multiclasse).
Précisez les performances en train et test pour les différentes configurations essayées. 

__Réponse__:


**Q6** Que se passe-t-il si vous changez de modalité de classification ? Faut-il réentraîner entièrement le modèle ? Quelles parties peuvent être conservées ?

__Réponse__ :

**Q7** Avez-vous des remarques concernant la classification d'objets par BoW (TP2) par rapport à la classification d'objets par CNN ?

__Réponse__ :